In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
include("../src/juliaEAM.jl")

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
using ASEconvert # use this PR:https://github.com/mfherbst/ASEconvert.jl/pull/17, Pkg.add(url="https://github.com/tjjarvinen/ASEconvert.jl.git", rev="atomscalculators")
using Unitful: Å, nm
using PythonCall
ENV["PYTHON"] = "/SNS/users/ccu/miniconda3/envs/analysis/bin/python"
using StaticArrays: SVector
using GLMakie
using Molly
using Zygote
using LinearAlgebra
import Interpolations:cubic_spline_interpolation, linear_interpolation, interpolate, BSpline, Cubic, scale, Line, OnGrid, extrapolate, Gridded, extrapolate, Flat
using DelimitedFiles
using UnitfulAtomic
import PeriodicTable
using Random
using ProgressMeter


  Activating new project at `~/Documents/ABCD_J/EAM`


In [2]:
## Basic atom informations
al_LatConst = 4.0495/10 # nm
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole

26.9815 u

## Define a system wo interaction 

In [3]:
## 1. Import ASE and other Python modules
# Import ASE and other Python modules as needed
ase = pyimport("ase")
ase_view = pyimport("ase.visualize")


function system_bulk(systemsize)

    # Build an (001) Al surface  
    # atoms_ase = ase.build.fcc100("Al", size=size, vacuum = al_LatConst*4*10)
    atoms_ase = ase.build.bulk("Al", "fcc", a=al_LatConst*10, cubic=true)
    current_atoms = atoms_ase
    for i in 1:systemsize[1]
        if i<2
            continue
        end
        current_atoms = ase.build.stack(current_atoms, atoms_ase, axis=0)
    end
    current_atoms_i = current_atoms
    for j in 1:systemsize[2]
        if j<2
            continue
        end
        current_atoms = ase.build.stack(current_atoms, current_atoms_i, axis=1)
    end
    current_atoms_i = current_atoms
    for j in 1:systemsize[3]
        if j<2
            continue
        end
        current_atoms = ase.build.stack(current_atoms, current_atoms_i, axis=2)
    end

    atoms_ase = current_atoms
    # atoms_ase.translate([al_LatConst*10/4,al_LatConst*10/4,0])
    atoms_ase.wrap()

    atoms_ase_cell = atoms_ase.get_cell()
    box_size = pyconvert(Array{Float64}, [atoms_ase_cell[x,x] for x in range(0,2)])*u"Å"
    # box_size[1]*=0.999 # distort the box size slightly
    # box_size[2]*=1.001 # distort the box size slightly

    # Build an Julia AtomsBase abstract 
    atoms_ab = pyconvert(AbstractSystem, atoms_ase)

    ## 4. Create Molly system
    ### 4.1 Convert atom positions to Molly's expected format (nanometers) and create Molly.Atom objects
    # Get atom positions from previously defined ASE system
    function get_positions(atoms_ase)
        positions = [(atom.position[1], atom.position[2], atom.position[3]) for atom in atoms_ase]
        return positions
    end

    # Convert each position from Ångströms to nanometers and ensure the conversion is applied element-wise.
    atom_positions = [SVector(uconvert(Å, pos[1]), 
        uconvert(Å, pos[2]), uconvert(Å, pos[3])) for pos in get_positions(atoms_ab)]

    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                            #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                            ) for i in 1:length(atom_positions)]
    return molly_atoms, atoms_ab, box_size, atom_positions, atoms_ase
end

system_bulk (generic function with 1 method)

## Define interaction

In [4]:
eam = EAM()
fname = "Al99.eam.alloy"
read_potential!(eam, fname)
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
end

## Define Simulator

In [5]:
# Define the MDSimulator structure
"""
In the constructor function MDSimulator, default values are provided for each of these fields. 
If you create a SteepestDescentMinimizer without specifying the types, default values 
will determine the types of the fields. For example, if you create a MDSimulator without specifying sigma, 
it will default to 0.1*u"Å", and S will be the type of this value.
"""
struct MDSimulator{S,W,D,L}
    sigma::S 
    W::W
    max_steps::Int
    dt::D
    log_stream::L
end


"""
    MDSimulator(; sigma=0.1*u"Å", W=1e-2*u"eV", max_steps=100, dt=1e-3u"ps",
                 log_stream=devnull)

Constructs a molecular dynamics simulator.

## Arguments
- `sigma`: The value of sigma in angstroms. Default is 0.1 Å.
- `W`: The value of W in electron volts. Default is 1e-2 eV.
- `max_steps`: The maximum number of simulation steps. Default is 100.
- `dt`: The time step size in picoseconds. Default is 1e-3 ps.
- `log_stream`: The stream to log simulation output. Default is `devnull`.

## Returns
- `MDSimulator`: The constructed molecular dynamics simulator.

"""
function MDSimulator(;
                        sigma=0.1*u"Å", W=1e-2*u"eV", max_steps=100, dt=1e-3u"ps",
                        log_stream=devnull)
    return MDSimulator(sigma, W, max_steps, dt, log_stream)
end

# Penalty function with Gaussuan form
"""
Returns a penalty function of system coordinate x with Gaussuan form
x:      System coordinate
x_0:    Reference system coordinate
sigma:  Spatial extent of the activation, per sqrt(degree of freedom)
W:      Strenth of activation, per degree of freedom
pbc:    Periodic boundary conditions
"""
function f_phi_p(x::Vector{SVector{3, typeof(1.0u"Å")}}, x_0, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"); nopenalty_atoms=[])
    N::Int = length(x)
    E_multiplier = ones(length(x))
    for atom in nopenalty_atoms
        E_multiplier[atom] = 0
    end
    
    sigma2_new = sigma^2
    EDSQ = (A, B) -> sum(sum(map(x -> x.^2, (A-B).*E_multiplier)))
    # phi_p = sum([W * exp(-EDSQ(x,c) / (2*sigma2_new)) for c in x_0]) # unit eV
    phi_p = 0.0u"eV"
    for c in x_0
        if EDSQ(x,c)<9*sigma2_new
            phi_p_individual = W * (exp(-EDSQ(x,c) / (2*sigma2_new)) - exp(-9/2))
            phi_p += phi_p_individual
        end
    end
    return phi_p
end

function grad_f_phi_p(x::Vector{SVector{3, typeof(1.0u"Å")}}, x_0, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"); nopenalty_atoms=[])
    N::Int = length(x)
    E_multiplier = ones(length(x))
    for atom in nopenalty_atoms
        E_multiplier[atom] = 0
    end

    sigma2_new = sigma^2
    EDSQ = (A, B) -> sum(sum(map(x -> x.^2, (A-B).*E_multiplier)))

    grad_phi_p = [(@SVector zeros(Float64,3))*u"eV/Å" for i in 1:N] # unit eV/Å
    for c in x_0
        if EDSQ(x,c)<9*sigma2_new
            grad_phi_p_individual = W * exp(-EDSQ(x,c) / (2*sigma2_new)) / (2*sigma2_new) * 2*(c-x).*E_multiplier
            grad_phi_p += grad_phi_p_individual
        end
    end
    return grad_phi_p
end

# Calculate the gradient of the penalty energy
function penalty_forces(sys::System, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"); nopenalty_atoms=[])
    # Function of the penalty energy for a given coordinate
    # f_phi_p_coords = x -> f_phi_p(x, penalty_coords, sigma, W)

    # Calculate the gradient of the penalty energy, The penalty force is the negative gradient of the penalty energy
    # penalty_fs = -gradient(f_phi_p_coords, sys.coords)[1] # unit eV/Å
    penalty_fs = -grad_f_phi_p(sys.coords, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # unit eV/Å

    return penalty_fs
end

# Define the forces function with penalty term
"""
Evaluate the forces acting on the system with penalty term
If there is no penalty term, the penalty_coords should be set to nothing, 
and return the forces identical to the original forces function
"""
function Molly.forces(sys::System, interaction::EAMInteractionJulia, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"), neighbors_all::Vector{Vector{Int}};
    n_threads::Integer=Threads.nthreads(), nopenalty_atoms=[]) 

    
    fs = interaction.f_forces(interaction.calculator, sys, neighbors_all)

    # Add penalty term to forces
    if penalty_coords != nothing
        fs += penalty_forces(sys, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # ev/Å
        # print(maximum(norm.(penalty_forces(sys, penalty_coords, sigma, W))),"\n")
    end
    return fs
end

"""
f_energy_phi(sys::System, sim::Simulator, penalty_coords)

Compute the total energy of the system `sys` including the potential energy contribution from the penalty coordinates.

# Arguments
- `sys::System`: The system for which the energy is to be computed.
- `sim::Simulator`: The simulator object containing simulation parameters.
- `penalty_coords`: The penalty coordinates used to calculate the potential energy contribution.

# Returns
- `E`: The total energy of the system with penalty terms.

"""
function f_energy_phi(sys::System, sim::MDSimulator, interaction::EAMInteractionJulia, penalty_coords, neighbors_all; nopenalty_atoms=[])
    E_phi = 0*u"eV"
    if penalty_coords!=nothing
        E_phi += f_phi_p(sys.coords, penalty_coords, sim.sigma, sim.W, nopenalty_atoms=nopenalty_atoms)
    end
    E = interaction.f_energy(interaction.calculator, sys, neighbors_all) + E_phi
    return E
end

"""
    lmpDumpWriter(file, timestep, sys, fname_dump)

Write system information to a LAMMPS dump file.

# Arguments
- `file`: The file object to write to.
- `timestep`: The current timestep of the simulation.
- `sys`: The Molly system object containing the coordinates and boundary information.
- `fname_dump`: The filename of the dump file.
"""
function lmpDumpWriter(file,timestep,sys,fname_dump)
    # open(fname_dump, "a") do file
    write(file, "ITEM: TIMESTEP\n")
    write(file, string(timestep)*"\n")
    write(file, "ITEM: NUMBER OF ATOMS\n")
    write(file, string(length(sys.coords))*"\n")
    write(file, "ITEM: BOX BOUNDS pp pp pp\n")
    write(file, "0 "*string(ustrip(sys.boundary[1]))*"\n")
    write(file, "0 "*string(ustrip(sys.boundary[2]))*"\n")
    write(file, "0 "*string(ustrip(sys.boundary[3]))*"\n")
    write(file, "ITEM: ATOMS id type xu yu zu\n")
    for (i_c, coord) in enumerate(sys.coords)
        write(file, string(i_c)*" 1 "*join(ustrip(coord)," ")*"\n")
    end
    # end
end

# Implement the simulate! function for MDSimulator

"""
    simulate!(sys::System, sim::MDSimulator, interaction::EAMInteractionJulia; 
               n_threads::Integer=Threads.nthreads(), run_loggers::Bool=true, fname::String="output_MD.txt", fname_dump="out.dump",
               neig_inteval::Int=1, loggers_inteval=1, dump_inteval = 1, start_dump = 1,
               mass::typeof(1.0u"u")=26.9815u"u", v_init = generate_velocity_distribution(0.0u"K", mass = 1.0u"u", num_atoms=length(sys.coords)))

Simulates the molecular dynamics (MD) of a system using the EAM (Embedded Atom Method) potential.

# Arguments
- `sys::System`: The system to be simulated.
- `sim::MDSimulator`: The MD simulator object.
- `interaction::EAMInteractionJulia`: The EAM interaction potential object.

# Optional Arguments
- `n_threads::Integer`: The number of threads to use for parallelization. Default is the number of available threads.
- `run_loggers::Bool`: Whether to run loggers during the simulation. Default is `true`.
- `fname::String`: The name of the output file to store simulation data. Default is "output_MD.txt".
- `fname_dump::String`: The name of the dump file to store simulation snapshots. Default is "out.dump".
- `neig_inteval::Int`: The interval at which to update the neighbor list. Default is 1.
- `loggers_inteval`: The interval at which to run loggers. Default is 1.
- `dump_inteval`: The interval at which to dump simulation snapshots. Default is 1.
- `start_dump`: The step number at which to start dumping snapshots. Default is 1.
- `mass::typeof(1.0u"u")`: The mass of the atoms in the system. Default is 26.9815u"u".
- `v_init`: The initial velocity distribution of the atoms. Default is generated using `generate_velocity_distribution`.

# Returns
- `sys::System`: The updated system after the simulation.

"""
function simulate!(sys::System, sim::MDSimulator, interaction::EAMInteractionJulia; 
                   n_threads::Integer=Threads.nthreads(), run_loggers::Bool=true, fname::String="output_MD.txt", fname_dump="out.dump",
                   neig_inteval::Int=1, loggers_inteval=1, dump_inteval = 1, start_dump = 1,
                   mass::typeof(1.0u"u")=26.9815u"u", v_init = generate_velocity_distribution(0.0u"K", mass = 1.0u"u", num_atoms=length(sys.coords)))
    n_steps = sim.max_steps
    # initialize
    neighbors_all = get_neighbors_all(sys)
    neighbors = find_neighbors(sys, sys.neighbor_finder; n_threads=n_threads)

    run_loggers!(sys, neighbors, 0, run_loggers; n_threads=n_threads)

    F = forces(sys, interaction, [], sim.sigma, sim.W, neighbors_all)
    accels_t = F/mass # force/mass
    accels_t_dt = [(@SVector zeros(3))*u"eV/Å/u" for i in 1:length(sys.coords)] # force/masses

    sys.velocities = v_init

    # open an empty output file
    open(fname, "w") do file
        write(file, "")
    end

    open(fname_dump, "w") do file
        write(file, "")
    end

    p = Progress(n_steps)
    open(fname_dump, "a") do file
        for step_n in 1:n_steps
            next!(p)

            sys.coords += sys.velocities .* sim.dt .+ ((accels_t .* sim.dt ^ 2) ./ 2)
            temperature = sum(sum(map(x -> x.^2, sys.velocities)))*mass/2/length(sys.coords)/3
            # print(ustrip(temperature)*1.2035,"\n")

            F = forces(sys, interaction, [], sim.sigma, sim.W, neighbors_all)
            accels_t_dt = F/mass # force/mass
            sys.velocities += ((accels_t .+ accels_t_dt) .* sim.dt / 2)

            if step_n % neig_inteval == 0
                neighbors_all = get_neighbors_all(sys)
            end

            accels_t = accels_t_dt



            if step_n % loggers_inteval == 0
                run_loggers!(sys, neighbors, step_n, run_loggers; n_threads=n_threads)
            end

            if step_n >= start_dump
                if step_n % dump_inteval == 0
                    lmpDumpWriter(file,step_n,molly_system,fname_dump)
                    # print("step ",step_n,"\n")
                end
            end
        end
    end

    return sys
end


simulate! (generic function with 1 method)

## Do simulation

In [6]:
eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces)
function initialize_system(loggers=(coords=CoordinateLogger(1),))
    molly_atoms, atoms_ab, box_size, atom_positions, _ = system_bulk((3,3,3))

    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(atom_positions)
    molly_atoms_init = copy(molly_atoms)
    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Al") for a in molly_atoms_init],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DistanceNeighborFinder(
    eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
    n_steps=1e3,
    dist_cutoff=9u"Å"),
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

## Alternatively, read from LAMMPS dump file
function lmpDumpReader(filename_dump)
    lines = readdlm(filename_dump, '\n', String) # read the files, split by new line
    function lines_to_list(lines) # convert the entries in lines to list
        data = []
        for line in lines
            push!(data, split(line))
        end
        return data
    end
    data = lines_to_list(lines)
    n_atoms = parse(Int,data[4][1])
    box_size = []
    box_origin = []
    for i_box in 1:3
        data_box = [parse(Float64,d) for d in data[i_box+5]]
        box_size_i = data_box[2]-data_box[1]
        push!(box_size,box_size_i*1u"Å")
        push!(box_origin,data_box[1])
    end
    id = zeros(Int,n_atoms)
    coords = []
    for i_atoms in 1:n_atoms
        data_atoms = [parse(Float64,d) for d in data[i_atoms+9]]
        id[i_atoms] = data_atoms[1]
        push!(coords, data_atoms[3:5]-box_origin)
    end
    coords_molly = [SVector{3}(c*1u"Å") for c in coords]
    return n_atoms, box_size, coords_molly
end

function initialize_system_dump(;loggers=(coords=CoordinateLogger(1),),filename_dump="")
    n_atoms, box_size, coords_molly = lmpDumpReader(filename_dump)
    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                    #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                    ) for i in 1:length(coords_molly)]
    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(coords_molly)
    molly_atoms_init = copy(molly_atoms)
    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Al") for a in molly_atoms_init],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DistanceNeighborFinder(
    eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
    n_steps=1e3,
    dist_cutoff=9u"Å"),
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

function generate_velocity_distribution(temperature, mass, num_atoms)
    kb = 8.617e-5u"eV*K^-1"  # Boltzmann constant in eV*K^-1
    sigma = ustrip(sqrt(kb * temperature / mass))  # Standard deviation of the velocity distribution

    velocities = []
    for i in 1:num_atoms
        vx = randn() * sigma
        vy = randn() * sigma
        vz = randn() * sigma
        push!(velocities, (@SVector [vx, vy, vz])*9814*u"m/s")
    end

    velocities_mean = sum(velocities)/length(molly_system.coords)
    velocities_mom = []
    for i in 1:num_atoms
        push!(velocities_mom, velocities[i]-velocities_mean)
    end

    return velocities_mom
end

generate_velocity_distribution (generic function with 1 method)

In [7]:
filename_dump = "./LAMMPS/out_initial.dump"

# initialize system
molly_system = initialize_system_dump(filename_dump = "./LAMMPS/out_initial_melt.dump")

# apply velocity
v0 = generate_velocity_distribution(298u"K", atom_mass, length(molly_system.coords))

# run simulation
simulator = MDSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=40000, dt=1e-3u"ps")
simulate!(molly_system, simulator, eamJulia; 
          fname="output_MD.txt", 
          fname_dump = "out_Al_test_J.dump",
          neig_inteval=200, loggers_inteval=1000, dump_inteval=1, start_dump=30000,
          mass = atom_mass,
          v_init = v0)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:34m


System with 108 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[12.495959062494398 Å, 12.495959062494398 Å, 12.495959062494398 Å])

## Visulize result

In [8]:
function visualize_wrap(coord_logger,
                    boundary,
                    out_filepath::AbstractString,
                    fig;
                    connections=Tuple{Int, Int}[],
                    connection_frames=[trues(length(connections)) for i in values(coord_logger)],
                    trails::Integer=0,
                    framerate::Integer=30,
                    color=:purple,
                    connection_color=:orange,
                    markersize=0.05,
                    linewidth=2.0,
                    transparency=true,
                    show_boundary::Bool=true,
                    boundary_linewidth=2.0,
                    boundary_color=:black,
                    az=1.275pi,
                    el=pi/8,
                    kwargs...)
    coords_start = first(values(coord_logger))
    dist_unit = unit(first(first(coords_start)))
    dims = n_dimensions(boundary)
    # fig = Figure()

    if dims == 3
        PointType = Point3f
        ax = Axis3(fig[1, 1], aspect=:data, azimuth=az, elevation=el)
        max_connection_dist = cbrt(box_volume(boundary)) / 2
    elseif dims == 2
        PointType = Point2f
        ax = Axis(fig[1, 1])
        ax.aspect = DataAspect()
        max_connection_dist = sqrt(box_volume(boundary)) / 2
    else
        throw(ArgumentError("found $dims dimensions but can only visualize 2 or 3 dimensions"))
    end

    positions = Observable(PointType.(ustrip_vec.(coords_start)))
    # scatter!(ax, positions; color=color, markersize=markersize, transparency=transparency,
    #             markerspace=:data, kwargs...)
    meshscatter!(ax, positions; color=color, markersize=markersize, transparency=transparency,
    kwargs...)

    if show_boundary
        lines!(
            ax,
            Molly.bounding_box_lines(boundary, dist_unit)...;
            color=boundary_color,
            linewidth=boundary_linewidth,
        )
    end

    connection_nodes = []
    for (ci, (i, j)) in enumerate(connections)
        # Don't display connected atoms that are likely connected over the box edge
        if first(connection_frames)[ci] && norm(coords_start[i] - coords_start[j]) < max_connection_dist
            if dims == 3
                push!(connection_nodes, Observable(PointType.(
                        ustrip.([coords_start[i][1], coords_start[j][1]]),
                        ustrip.([coords_start[i][2], coords_start[j][2]]),
                        ustrip.([coords_start[i][3], coords_start[j][3]]))))
            elseif dims == 2
                push!(connection_nodes, Observable(PointType.(
                        ustrip.([coords_start[i][1], coords_start[j][1]]),
                        ustrip.([coords_start[i][2], coords_start[j][2]]))))
            end
        else
            if dims == 3
                push!(connection_nodes, Observable(PointType.([0.0, 0.0], [0.0, 0.0],
                                                        [0.0, 0.0])))
            elseif dims == 2
                push!(connection_nodes, Observable(PointType.([0.0, 0.0], [0.0, 0.0])))
            end
        end
    end
    for (ci, cn) in enumerate(connection_nodes)
        lines!(ax, cn;
                color=isa(connection_color, AbstractArray) ? connection_color[ci] : connection_color,
                linewidth=isa(linewidth, AbstractArray) ? linewidth[ci] : linewidth,
                transparency=transparency)
    end

    trail_positions = []
    for trail_i in 1:trails
        push!(trail_positions, Observable(PointType.(ustrip_vec.(coords_start))))
        col = parse.(Colorant, color)
        alpha = 1 - (trail_i / (trails + 1))
        alpha_col = RGBA.(red.(col), green.(col), blue.(col), alpha)
        # scatter!(ax, trail_positions[end]; color=alpha_col,  markersize=markersize,
        #             transparency=transparency, markerspace=:data, kwargs...)
        meshscatter!(ax, trail_positions[end]; color=alpha_col, markersize=markersize, transparency=transparency,
                    kwargs...)
    end

    boundary_conv = ustrip.(dist_unit, Molly.cubic_bounding_box(boundary))
    xlims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 1))
    ylims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 2))
    dims == 3 && zlims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 3))

    GLMakie.record(fig, out_filepath, eachindex(values(coord_logger)); framerate=framerate) do frame_i
        coords = values(coord_logger)[frame_i]
        # coords = wrap_coords.(coords, (boundary,))
        u_coords = unit(coords[1][1])
        coords = [ustrip(c) for c in coords]*uconvert(u"nm", 1*u_coords)
        # print(coords[1][1])

        for (ci, (i, j)) in enumerate(connections)
            if connection_frames[frame_i][ci] && norm(coords[i] - coords[j]) < max_connection_dist
                if dims == 3
                    connection_nodes[ci][] = PointType.(
                                ustrip.([coords[i][1], coords[j][1]]),
                                ustrip.([coords[i][2], coords[j][2]]),
                                ustrip.([coords[i][3], coords[j][3]]))
                elseif dims == 2
                    connection_nodes[ci][] = PointType.(
                                ustrip.([coords[i][1], coords[j][1]]),
                                ustrip.([coords[i][2], coords[j][2]]))
                end
            else
                if dims == 3
                    connection_nodes[ci][] = PointType.([0.0, 0.0], [0.0, 0.0],
                                                        [0.0, 0.0])
                elseif dims == 2
                    connection_nodes[ci][] = PointType.([0.0, 0.0], [0.0, 0.0])
                end
            end
        end

        positions[] = PointType.(ustrip_vec.(coords))
        for (trail_i, trail_position) in enumerate(trail_positions)
            trail_position[] = PointType.(ustrip_vec.(values(coord_logger)[max(frame_i - trail_i, 1)]))
        end
    end
end


visualize_wrap (generic function with 1 method)

In [9]:
## visualize
using ColorSchemes
using Colors

# # Define the color gradient
# color_0 = colorant"#000000"
# color_1 = colorant"#FFFFFF"
# color_gradient = ColorScheme(range(color_0, color_1, length=100))

colors = []
coords_z = [c[3] for c in molly_system.coords]
z_max = maximum(coords_z[1:end-1])
z_min = minimum(coords_z[1:end-1])
for (index, value) in enumerate(molly_system.coords)
    # z_component = value[3]
    # z_ratio = 2*(z_component-z_min)/(z_max-z_min)-1
    # color = color_gradient[z_ratio]
    color = colorant"#FF5555"
    push!(colors, color)
end
framerate = 20
molly_atoms, atoms_ab, box_size, atom_positions = system_bulk((3,3,3))

# Specify boundary condition
boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])
fig = Figure(size = (400, 400))
visualize_wrap(molly_system.loggers.coords, boundary_condition, "test_Julia_MD.gif", fig; 
                markersize=0.1, color=colors, az=-5*pi/12, framerate=framerate, transparency=false)

"test_Julia_MD.gif"